In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

# Define a mapping of object class numbers to corresponding labels
class_mapping = {
    0: "car_front",
    1: "car_back",
    2: "car_side"
}
label_to_int = {v: k for k, v in class_mapping.items()}

# Set the folder path where the files are located
folder_path = "/Users/ariehawazie/Desktop/Vehicle Orientation/vehicle-orientation-5"

# Get the names of all files in the directory
file_names = os.listdir(folder_path)

# Filter out the file names for .txt and .jpg files
txt_files = [file for file in file_names if file.endswith(".txt")]
jpg_files = [file for file in file_names if file.endswith(".jpg")]

data = []
desired_size = (224, 224)  # Set the desired size for the resized images

# Process images in batches
batch_size = 100  # Number of images to process in each batch
num_batches = len(jpg_files) // batch_size + 1
orginal_image_width = 1920
orginal_image_height = 1080

for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(jpg_files))

    # Loop through each .jpg file in the batch
    for jpg_file in jpg_files[start_idx:end_idx]:
        base_name = os.path.splitext(jpg_file)[0]  # remove the .jpg extension
        txt_file = base_name + ".txt"  # create the corresponding .txt file name

        # Check if the corresponding .txt file exists
        if txt_file in txt_files:
            # Open and read the contents of the .txt file
            with open(os.path.join(folder_path, txt_file), "r") as file:
                lines = file.readlines()

            objects = []
            # Loop through each line in the .txt file
            for line in lines:
                line_parts = line.strip().split()
                object_class = int(line_parts[0])

                # Check if the object class indicates a car (numbers 0-2)
                if object_class in [0, 1, 2]:
                    # Map the object class number to the corresponding label
                    class_label = class_mapping[object_class]

                    # Create a dictionary representing the object with its properties
                    objects.append({"class": class_label})

            # Check if there are any valid objects
            if objects:
                # Open the corresponding .jpg image
                with Image.open(os.path.join(folder_path, jpg_file)) as img:
                    # Resize the image
                    resized_image = img.resize(desired_size)

                    # Convert the image to a NumPy array
                    resized_image_array = np.array(resized_image)
                    resized_image_array = resized_image_array.astype(np.float32)

                    # Normalize the pixel values
                    resized_image_array /= 255.0

                # Create a dictionary representing the data for this resized image
                data.append({"objects": objects, "image": resized_image_array})

# Convert data to separate lists for images and labels
images = [item["image"] for item in data]
labels = [label_to_int[item["objects"][0]['class']] for item in data]  # Note that each object now only has one class

# One-hot encode the labels
labels = to_categorical(labels)

# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convert the lists to NumPy arrays
train_images = np.array(train_images)
val_images = np.array(val_images)
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

# Define the model architecture
IMG_SIZE = 224
NUM_CLASSES = 3

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

# Build the model
model = build_model(NUM_CLASSES)

# Print the model summary
model.summary()

# Define number of epochs
epochs = 20

# Train the model
history = model.fit(train_images, train_labels, epochs=epochs, validation_data=(val_images, val_labels), verbose=1)

# Now we'll unfreeze some layers and do a second round of training.
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

unfreeze_model(model)

fine_tuning_epochs = 20
history_fine_tune = model.fit(train_images, train_labels, epochs=fine_tuning_epochs, validation_data=(val_images, val_labels), verbose=2)

# Generate predictions for the validation set
val_predictions = model.predict(val_images)

# Convert one-hot encoded predictions and labels back to integers
val_labels_integers = np.argmax(val_labels, axis=1)
val_predictions_integers = np.argmax(val_predictions, axis=1)

# Calculate and print classification report
report = classification_report(val_labels_integers, val_predictions_integers, target_names=class_mapping.values())
print(report)



Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_3 (Rescaling)        (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 rescaling_4 (Rescaling)        (None, 224, 224, 3)  0           ['rescaling_3[0][0]']            
                                                                                                  
 normalization_1 (Normalization  (None, 224, 224, 3)  7          ['rescaling_4[0][0]'] 

/Users/ariehawazie/mambaforge/envs/Data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ariehawazie/mambaforge/envs/Data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ariehawazie/mambaforge/envs/Data/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

In [7]:
# Define the directory where you want to save the model
dir_name = "/Users/ariehawazie/Desktop/Vehicle Orientation/"

# Save the model weights in this directory
model.save_weights(os.path.join(dir_name, 'my_model_weights.h5'))


In [10]:
from PIL import Image
import numpy as np

# Define the path of your test image
image_path = "/Users/ariehawazie/Desktop/Vehicle Orientation/Test/Test1.jpg"

# Load the image and convert it to RGB
img = Image.open(image_path).convert('RGB')

# Resize the image to match the input shape of your model
img = img.resize((224, 224))

# Convert image to numpy array
img_array = np.array(img)

# Scale the image data to range [0, 1]
img_array = img_array.astype(np.float32) / 255.0

# Add an extra dimension because the model expects batches of images
img_array = np.expand_dims(img_array, axis=0)

# Predict the class of the image using your model
predictions = model.predict(img_array)

# The output of the model is a vector of "class probabilities",
# so we'll get the index of the highest probability class as the predicted class
predicted_class_idx = np.argmax(predictions[0])

# Get the name of the predicted class
predicted_class = class_mapping[predicted_class_idx]

print(f'The model predicts that the image is a {predicted_class}.')


1/1 [==============================] - 0s 165ms/step
The model predicts that the image is a car_front.


In [16]:
# Define the path of your test image
image_path = "/Users/ariehawazie/Desktop/Vehicle Orientation/Test/Test5.jpg"

# Load the image and convert it to RGB
img = Image.open(image_path).convert('RGB')

# Resize the image to match the input shape of your model
img = img.resize((224, 224))

# Convert image to numpy array
img_array = np.array(img)

# Scale the image data to range [0, 1]
img_array = img_array.astype(np.float32) / 255.0

# Add an extra dimension because the model expects batches of images
img_array = np.expand_dims(img_array, axis=0)

# Predict the class of the image using your model
predictions = model.predict(img_array)

# The output of the model is a vector of "class probabilities",
# so we'll get the index of the highest probability class as the predicted class
predicted_class_idx = np.argmax(predictions[0])

# Get the name of the predicted class
predicted_class = class_mapping[predicted_class_idx]

print(f'The model predicts that the image is a {predicted_class}.')

1/1 [==============================] - 0s 73ms/step
The model predicts that the image is a car_front.


In [17]:
# Define the model architecture (this should be exactly the same as when you originally created the model)
model = build_model(NUM_CLASSES)

#Then, to use the model again, you'll have to recreate the model architecture and load the weights into it:

# Load the weights into the model
model.load_weights(os.path.join(dir_name, 'my_model_weights.h5'))

#Remember, when using this approach, you must ensure that the model architecture you define when loading the weights is exactly the same as the architecture you used when saving the weights.

In [18]:
# Define the path of your test image
image_path = "/Users/ariehawazie/Desktop/Vehicle Orientation/Test/Test3.jpg"

# Load the image and convert it to RGB
img = Image.open(image_path).convert('RGB')

# Resize the image to match the input shape of your model
img = img.resize((224, 224))

# Convert image to numpy array
img_array = np.array(img)

# Scale the image data to range [0, 1]
img_array = img_array.astype(np.float32) / 255.0

# Add an extra dimension because the model expects batches of images
img_array = np.expand_dims(img_array, axis=0)

# Predict the class of the image using your model
predictions = model.predict(img_array)

# The output of the model is a vector of "class probabilities",
# so we'll get the index of the highest probability class as the predicted class
predicted_class_idx = np.argmax(predictions[0])

# Get the name of the predicted class
predicted_class = class_mapping[predicted_class_idx]

print(f'The model predicts that the image is a {predicted_class}.')

1/1 [==============================] - 1s 1s/step
The model predicts that the image is a car_front.
